## 学到的神奇操作

In [ ]:
def printlist(info_list):
    '''结果输出，也可使用Pandas'''
    tplt = "{:^5}\t{:^10}\t{:^50}"
    print(tplt.format("序号", "商品价格", "商品名称"))
    count = 0
    for info in info_list:
        count += 1
        print(tplt.format(count, info[0], info[1]))

In [ ]:
printGoodList(info_list)

## 正则表达式
### 作用
- 测试字符串内的模式。
    例如，可以测试输入字符串，以查看字符串内是否出现电话号码模式或信用卡号码模式。这称为数据验证。
- 替换文本。
    可以使用正则表达式来识别文档中的特定文本，完全删除该文本或者用其他文本替换它。
- 基于模式匹配从字符串中提取子字符串。
    可以查找文档内或输入域内特定的文本。

### 语法
```python
1. . 表示任何单个字符
2. [ ] 字符集，对单个字符给出取值范围 ，如[abc]表示a、b、c，[a‐z]表示a到z单个字符
3. [^ ] 非字符集，对单个字符给出排除范围 ，如[^abc]表示非a或b或c的单个字符
4. * 前一个字符0次或无限次扩展，如abc* 表示 ab、abc、abcc、abccc等
5. + 前一个字符1次或无限次扩展 ，如abc+ 表示 abc、abcc、abccc等
6. ? 前一个字符0次或1次扩展 ，如abc? 表示 ab、abc
7. | 左右表达式任意一个 ，如abc|def 表示 abc、def
8. {m} 扩展前一个字符m次 ，如ab{2}c表示abbc
9. {m,n} 扩展前一个字符m至n次（含n） ，如ab{1,2}c表示abc、abbc
10. ^ 匹配字符串开头 ，如^abc表示abc且在一个字符串的开头
11. $ 匹配字符串结尾 ，如abc$表示abc且在一个字符串的结尾
12. ( ) 分组标记，内部只能使用 | 操作符 ，如(abc)表示abc，(abc|def)表示abc、def
13. \d 数字，等价于[0‐9]
14. \w 单词字符，等价于[A‐Za‐z0‐9_]
```

### 主要功能函数
- re.search() 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象
    - re.search(pattern, string, flags=0)
- re.match() 从一个字符串的开始位置起匹配正则表达式，返回match对象
    - re.match(pattern, string, flags=0)
- re.findall() 搜索字符串，以列表类型返回全部能匹配的子串
    - re.findall(pattern, string, flags=0)
- re.split() 将一个字符串按照正则表达式匹配结果进行分割，返回列表类型
    - re.split(pattern, string, maxsplit=0, flags=0)
- re.finditer() 搜索字符串，返回一个匹配结果的迭代类型，每个迭代元素是match对象
    - re.finditer(pattern, string, flags=0)
- re.sub() 在一个字符串中替换所有匹配正则表达式的子串，返回替换后的字符串
    - re.sub(pattern, repl, string, count=0, flags=0)
    - flags : 正则表达式使用时的控制标记：
        - re.I --> re.IGNORECASE : 忽略正则表达式的大小写，\[A‐Z]能够匹配小写字符
        - re.M --> re.MULTILINE : 正则表达式中的^操作符能够将给定字符串的每行当作匹配开始
        - re.S --> re.DOTALL : 正则表达式中的.操作符能够匹配所有字符，默认匹配除换行外的所有字符

### 补充
**re库的另一种等价用法（编译）**

regex = re.compile(pattern, flags=0)：将正则表达式的字符串形式编译成正则表达式对象

**re 库的贪婪匹配和最小匹配**
- .* Re库默认采用贪婪匹配，即输出匹配最长的子串
- *? 只要长度输出可能不同的，都可以通过在操作符后增加?变成最小匹配

### 实战：淘宝商品比价定向爬虫

url = ‘https://s.taobao.com/search?q=书包&js=1&stats_click=search_radio_all%25’

In [180]:
import requests
import re

In [181]:
def gethtml(url,page):
    '''发送请求，获得相应数据'''
    headers = {
        'user-agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36',
        'cookie':
        'cna=LUTNFp98HgICAbfQbqG69yTN; miid=883322181641661587; thw=cn; t=18d9014034c3d7cdbf22e0e0a36389cc; sgcookie=E29N6vJEFyFHZnS8l4FeN; uc3=nk2=2mxIgRGsf84%3D&id2=UUGk1yyTQ3niFw%3D%3D&lg2=U%2BGCWk%2F75gdr5Q%3D%3D&vt3=F8dBxGR2VDrLz2TMKBk%3D; lgc=%5Cu5E7F%5Cu6155%5Cu4E00%5Cu541B; uc4=nk4=0%402FZXUWtnKEf9cHHvYZrDMUKxlA%3D%3D&id4=0%40U2OT53LpZWcybOSAnCSqgDRG2Zgf; tracknick=%5Cu5E7F%5Cu6155%5Cu4E00%5Cu541B; _cc_=V32FPkk%2Fhw%3D%3D; enc=4JieTg91uCqW8w8tN0SJVVgFDUlDxb%2BJyzRMdsXoYioud%2BuDK02H45acsdFjm7I7zu8z9KIuVNp3gu5OeslVHw%3D%3D; tfstk=cco5BAfVG_fSEYmP0X94zhw9OrqAapAbnUN-FqhdW5P6V_D7MsDhb5ZkQ2fcDPef.; mt=ci=9_1; hng=CN%7Czh-CN%7CCNY%7C156; JSESSIONID=DB7B892FE15A99AAF4B17C5B88C76EBB; uc1=cookie14=UoTUPclonZEnOA%3D%3D; cookie2=59a6025621adae4fa855dcd33cdbf335; v=0; _tb_token_=a13ee3be7e75; l=eBNr8SXVQUiUqSDbBOfZnurza77TOIRAguPzaNbMiT5P_yfH5B1OWZjY0ELMCnGVh62DJ3yAaP04BeYBqCmWfdW22j-laOkmn; isg=BPj4FZyr0vusyD6WOERLsp6SyaaKYVzrkxNoTjJpaTPmTZg323FMexxnBUV9HRTD'
    }
    params = {
        'q':'书包',
        's':44*page
        
    }
    try:
        res = requests.get(url,params=params,headers=headers)
        res.raise_for_status()
        res.encoding = 'utf-8'
        return res.text
    except:
        return '产生异常'

**补充:  .*?的使用**

后边多一个？表示懒惰模式。必须跟在*或者+后边用

如：`<img src="test.jpg" width="60px" height="80px"/>`

如果用正则匹配src中内容非懒惰模式匹配

src=".*"

匹配结果是：src="test.jpg" width="60px" height="80px"

意思是从="往后匹配，直到最后一个"匹配结束

懒惰模式正则：

src=".*?"

结果：src="test.jpg"

因为匹配到第一个"就结束了一次匹配。不会继续向后匹配。因为他懒惰嘛。

.表示除\n之外的任意字符

*表示匹配0-无穷

+表示匹配1-无穷


In [182]:
def parsehtml(html):
    '''通过正则解析数据'''
    glist = []
    price_list = re.findall(r'view_price":"(.*?)","view_fee', html)
    name_list = re.findall(r'raw_title":"(.*?)","pic_url":', html)
    for i, j in zip(price_list, name_list):
        glist.append([i, j])
#         print(glist)
    return glist

In [183]:
def printlist(info_list):
    '''结果输出，也可使用Pandas'''
    tplt = "{:^5}\t{:^10}\t{:^50}"
    print(tplt.format("序号", "商品价格", "商品名称"))
    count = 0
    for info in info_list:
        count += 1
        print(tplt.format(count, info[0], info[1]))

In [184]:
url = 'https://s.taobao.com/search?'
page = 3
info_list = []

In [185]:
for i in range(page):
    html = gethtml(url,page)
    info_list.extend(parsehtml(html))  # extend() 函数用于在列表末尾一次性追加另一个序列中的多个值（用新列表扩展原来的列表）。
printlist(info_list)
print('已完成')

序号  	   商品价格   	                       商品名称                       
  1  	  189.00  	         不莱玫包包2020新款网红百搭时尚托特包手提单肩包女大容量书包潮         
  2  	  179.00  	        不莱玫2020新款迪士尼书包时尚复古ins风小背包女双肩包迷你小包         
  3  	  76.00   	          儿童书包小学生女一二三到六年级韩版减负护脊男孩轻便防水书包           
  4  	  115.00  	        KK剑桥树书包小学生男孩一二三到六年级儿童6-12岁男童双肩包减负         
  5  	  378.00  	      tigerfamily书包小学生男孩1-3-6年级 初高中儿童护脊减负背包女      
  6  	  179.00  	         安踏双肩包2020潮牌大容量旅行背包男休闲简约学生书包运动背包男         
  7  	  299.00  	      Viney真皮双肩包2020年新款韩版百搭书包韩版原宿ulzzang双肩背包      
  8  	  139.00  	        阿迪达斯学生书包男女包初中高中大学生电脑包运动双肩背包FI7968         
  9  	  59.80   	          商务背包男士双肩包韩版潮流旅行休闲女中学生书包简约时尚电脑包          
 10  	  858.00  	         韩国直邮 PETER JENSEN彼得詹森经典双肩背包儿童书包          
 11  	  249.00  	       Nike耐克2020年新款Tanjun儿童双肩背包男女休闲书包手提包拎包       
 12  	 1099.00  	        Fion/菲安妮新款时尚包包 女士旅行旅游背包 绣花双肩包潮牌书包         
 13  	  289.00  	      Samsonite/新秀丽儿童背包可爱卡通动物造型学生书包3D双肩包 U22      
 14  	  469.00  	       Doughnut Macaroon粉彩背包旅行书包